# Init

In [1]:

import requests
import pandas as pd
import time
import os
import plotly.graph_objects as go
from datetime import datetime, timedelta

import business.utils.trading_signals as ts
import business.utils.trading_indicators as ti


base_start_time_1m = int(pd.Timestamp("2019-01-01").timestamp() * 1000)
base_start_time_other = int(pd.Timestamp("2015-01-01").timestamp() * 1000)

interval_to_ms = {
    "1m": 60_000,
    "3m": 3 * 60_000,
    "5m": 5 * 60_000,
    "15m": 15 * 60_000,
    "30m": 30 * 60_000,
    "1h": 60 * 60_000,
    "2h": 2 * 60 * 60_000,
    "4h": 4 * 60 * 60_000,
    "6h": 6 * 60 * 60_000,
    "8h": 8 * 60 * 60_000,
    "12h": 12 * 60 * 60_000,
    "1d": 24 * 60 * 60_000,
    "3d": 3 * 24 * 60 * 60_000,
    "1w": 7 * 24 * 60 * 60_000,
    "1M": 30 * 24 * 60 * 60_000  # approximate month (30 days)
}


timeframe_array = [
                '1m','15m','1h','4h', '1d', '1w',
                #'1M'
                ]
symbol_array = ['BTCUSDT', 'ETHUSDT', 'SOLUSDT','ICPUSDT', 'AVAXUSDT']

In [2]:
def get_binance_klines(symbol, interval, start_time, end_time=None, limit=1000):
    url = "https://api.binance.com/api/v3/klines"
    params = {
        "symbol": symbol,
        "interval": interval,
        "startTime": start_time,
        "limit": limit
    }
    if end_time:
        params["endTime"] = end_time
    response = requests.get(url, params=params)
    data = response.json()
    return data

In [3]:
def get_existing_data_path(symbol, interval):
    """Return the exact file path if the CSV exists (case-sensitive for intervals)."""
    filename = f"data/exports/{symbol}_{interval}_data.csv"
    return filename if os.path.exists(filename) else None

In [4]:
def get_last_timestamp_from_csv(filepath):
    """Retrieve the last timestamp from an existing CSV file."""
    df = pd.read_csv(filepath, usecols=["timestamp"])
    if df.empty:
        return None
    last_timestamp = df["timestamp"].iloc[-1]
    return int(pd.Timestamp(last_timestamp).timestamp() * 1000)

In [5]:

def export_historical_data(symbol, interval, start_time):
    if interval not in interval_to_ms:
        raise ValueError(f"Interval {interval} not supported.")

    delta = interval_to_ms[interval]

    all_data = []
    start_overall = time.time()
    iteration_count = 0

    while True:
        iteration_count += 1
        klines = get_binance_klines(symbol, interval, start_time, limit=1000)
        if not klines:
            print("No more data returned from Binance.")
            break

        all_data += klines
        # Advance start_time by the appropriate millisecond delta for the interval.
        start_time = klines[-1][0] + delta

        elapsed = time.time() - start_overall
        latest_ts = klines[-1][0]
        latest_dt = pd.to_datetime(latest_ts, unit='ms')
        print(f"Iteration {iteration_count}: Latest timestamp: {latest_ts} ({latest_dt}), "
              f"Total records: {len(all_data)}, Elapsed time: {elapsed:.2f} sec")

        # If fewer than 'limit' records are returned, assume we've reached the end.
        if len(klines) < 1000:
            print(f"Iteration {iteration_count}: Last batch retrieved with {len(klines)} records. Ending extraction.")
            break

        # Respect Binance rate limits.
        time.sleep(0.5)

    # Convert collected data into a DataFrame.
    df = pd.DataFrame(all_data, columns=[
        "timestamp", "open", "high", "low", "close", "volume",
        "close_time", "quote_asset_volume", "number_of_trades",
        "taker_buy_base_asset_volume", "taker_buy_quote_asset_volume", "ignore"
    ])
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit='ms')
    df.set_index("timestamp", inplace=True)

    print("Data extraction complete. Sample data:")
    print(df.head())

    # Export to CSV.
    df.to_csv(f'data/exports/{symbol}_{interval}_data.csv', index=True)

In [6]:
def append_new_data(symbol, interval, start_time, csv_path):
    """Append new data to an existing CSV file and save."""
    # Fetch new data from Binance
    new_data = []
    iteration_count = 0
    start_overall = time.time()

    while True:
        iteration_count += 1
        klines = get_binance_klines(symbol, interval, start_time, limit=1000)
        if not klines:
            print(f"{symbol} {interval}: No more data from Binance.")
            break

        new_data += klines
        start_time = klines[-1][0] + interval_to_ms[interval]

        elapsed = time.time() - start_overall
        latest_ts = klines[-1][0]
        latest_dt = pd.to_datetime(latest_ts, unit='ms')
        print(f"{symbol} {interval} | Iteration {iteration_count}: Latest timestamp: {latest_dt}, "
              f"Total new records: {len(new_data)}, Elapsed: {elapsed:.2f}s")

        if len(klines) < 1000:
            print(f"{symbol} {interval}: Last batch retrieved with {len(klines)} records. Reached end.")
            break

        time.sleep(0.5)  # Respect Binance rate limits

    if not new_data:
        print(f"{symbol} {interval}: No new data to append.")
        return

    # Convert collected data into a DataFrame
    df_new = pd.DataFrame(new_data, columns=[
        "timestamp", "open", "high", "low", "close", "volume",
        "close_time", "quote_asset_volume", "number_of_trades",
        "taker_buy_base_asset_volume", "taker_buy_quote_asset_volume", "ignore"
    ])
    df_new["timestamp"] = pd.to_datetime(df_new["timestamp"], unit='ms')

    # Load existing data
    df_existing = pd.read_csv(csv_path)
    df_existing["timestamp"] = pd.to_datetime(df_existing["timestamp"])

    # Combine and remove duplicates
    df_combined = pd.concat([df_existing, df_new]).drop_duplicates(subset=["timestamp"]).sort_values(by="timestamp")

    # Save updated CSV
    df_combined.to_csv(csv_path, index=False)
    print(f"{symbol} {interval}: Data appended and saved. Total records: {len(df_combined)}")


In [7]:
def export_or_update(symbol, interval):
    """Check if export exists, update if yes, otherwise download from scratch."""
    csv_path = get_existing_data_path(symbol, interval)

    if csv_path:
        print(f"✅ {symbol} {interval}: Existing CSV found. Checking for updates...")
        last_timestamp = get_last_timestamp_from_csv(csv_path)
        if last_timestamp:
            print(f"📅 Last recorded timestamp: {pd.to_datetime(last_timestamp, unit='ms')}")
            append_new_data(symbol, interval, last_timestamp, csv_path)
        else:
            start_time = base_start_time_1m if interval == "1m" else base_start_time_other
            print(f"⚠️ {symbol} {interval}: CSV is empty, starting from {pd.to_datetime(start_time, unit='ms')}...")
            export_historical_data(symbol, interval, start_time)
    else:
        start_time = base_start_time_1m if interval == "1m" else base_start_time_other
        print(f"🚀 {symbol} {interval}: No existing data. Starting from {pd.to_datetime(start_time, unit='ms')}...")
        export_historical_data(symbol, interval, start_time)

# Extract

In [27]:
for symbol in symbol_array:
        for interval in timeframe_array:
            print(f"📊 Processing {symbol} {interval}")
            export_or_update(symbol, interval)
            time.sleep(2)

📊 Processing BTCUSDT 1m
✅ BTCUSDT 1m: Existing CSV found. Checking for updates...
📅 Last recorded timestamp: 2025-02-15 12:27:00
BTCUSDT 1m | Iteration 1: Latest timestamp: 2025-02-15 15:55:00, Total new records: 209, Elapsed: 1.00s
BTCUSDT 1m: Last batch retrieved with 209 records. Reached end.


KeyboardInterrupt: 

# Insert higher timeframes

In [8]:
data_folder = 'data/exports'

timeframe_array = ['1m','15m','1h','4h', '1d', '1w']
timeframe_mapping = {'1m': '1min', '15m': '15min', '1h': '1h', '4h': '4h', '1d': '1D', '1w': '7D'}

In [10]:
def merge_timeframes():
    data = {}
    
    # Load datasets into a dictionary
    for symbol in [
                # 'BTCUSDT','ETHUSDT',
                #'SOLUSDT',
                #'ICPUSDT',
                'AVAXUSDT'
            ]:
        for timeframe in timeframe_array:
            file_path = os.path.join(data_folder, f'{symbol}_{timeframe}_data.csv')
            if os.path.exists(file_path):
                df = pd.read_csv(file_path, parse_dates=['timestamp'])
                data[(symbol, timeframe)] = df.set_index('timestamp')
    
    for (symbol, timeframe), df in data.items():
        higher_timeframes = [tf for tf in timeframe_array 
                             if pd.Timedelta(timeframe_mapping[tf]) > pd.Timedelta(timeframe_mapping[timeframe])]
        
        for ht in higher_timeframes:
            if (symbol, ht) in data:
                ht_df = data[(symbol, ht)].reindex(df.index, method='ffill')
                df[f'{ht}_open'] = ht_df['open']
                df[f'{ht}_high'] = ht_df['high']
                df[f'{ht}_low'] = ht_df['low']
                # Use the current close from the lower timeframe (since we don't know the higher timeframe's final close)
                df[f'{ht}_close'] = df['close']
        
        output_path = os.path.join(data_folder, f'{symbol}_{timeframe}_data.csv')
        df.reset_index().to_csv(output_path, index=False)
        print(f'Saved merged file: {output_path}')

In [9]:
def check_missing_intervals():
    for timeframe, freq in timeframe_mapping.items():
        for symbol in symbol_array:
            file_path = os.path.join(data_folder, f'{symbol}_{timeframe}_data.csv')
            if os.path.exists(file_path):
                df = pd.read_csv(file_path, parse_dates=['timestamp'])
                df = df.set_index('timestamp')
                
                # Check for duplicates
                duplicates = df.index.duplicated().sum()
                if duplicates > 0:
                    print(f'{symbol}_{timeframe}: {duplicates} duplicate timestamps found.')
                
                # Check for missing intervals
                all_times = pd.date_range(start=df.index.min(), end=df.index.max(), freq=freq)
                missing_times = all_times.difference(df.index)
                if not missing_times.empty:
                    print(f'{symbol}_{timeframe}: {len(missing_times)} missing timestamps.')
                else:
                    print(f'{symbol}_{timeframe}: No missing timestamps.')

In [11]:
def find_missing_timestamps(df_name):
    # Extract symbol and timeframe from dataframe name (e.g., 'BTCUSDT_1m')
    symbol_timeframe = df_name.replace('_data', '').strip()  # Clean suffix if any
    symbol, timeframe = symbol_timeframe.split('_')
    
    
    file_path = f'data/exports/{df_name}.csv'
    if not os.path.exists(file_path):
        print(f"File {file_path} not found.")
        return None

    df = pd.read_csv(file_path, parse_dates=['timestamp']).set_index('timestamp')

    if timeframe not in timeframe_mapping:
        print(f"Timeframe {timeframe} not recognized.")
        return None

    # Create a full range of timestamps for the timeframe
    freq = timeframe_mapping[timeframe]
    full_range = pd.date_range(start=df.index.min(), end=df.index.max(), freq=freq)

    # Find missing timestamps
    missing_timestamps = full_range.difference(df.index)

    # Create a dataframe for the missing timestamps
    missing_df = pd.DataFrame({'timestamp': missing_timestamps})
    
    if missing_df.empty:
        print(f"No missing timestamps for {df_name}.")
    else:
        print(f"Found {len(missing_df)} missing timestamps for {df_name}.")

    return missing_df

In [12]:
# merge_timeframes()
check_missing_intervals()

BTCUSDT_1m: 3357 missing timestamps.
ETHUSDT_1m: 4111 missing timestamps.
SOLUSDT_1m: 1444 missing timestamps.
ICPUSDT_1m: 470 missing timestamps.
AVAXUSDT_1m: 1445 missing timestamps.
BTCUSDT_15m: 565 missing timestamps.
ETHUSDT_15m: 565 missing timestamps.
SOLUSDT_15m: 94 missing timestamps.
ICPUSDT_15m: 31 missing timestamps.
AVAXUSDT_15m: 94 missing timestamps.
BTCUSDT_1h: 128 missing timestamps.
ETHUSDT_1h: 128 missing timestamps.
SOLUSDT_1h: 20 missing timestamps.
ICPUSDT_1h: 7 missing timestamps.
AVAXUSDT_1h: 20 missing timestamps.
BTCUSDT_4h: 16 missing timestamps.
ETHUSDT_4h: 16 missing timestamps.
SOLUSDT_4h: No missing timestamps.
ICPUSDT_4h: No missing timestamps.
AVAXUSDT_4h: No missing timestamps.
BTCUSDT_1d: No missing timestamps.
ETHUSDT_1d: No missing timestamps.
SOLUSDT_1d: No missing timestamps.
ICPUSDT_1d: No missing timestamps.
AVAXUSDT_1d: No missing timestamps.
BTCUSDT_1w: No missing timestamps.
ETHUSDT_1w: No missing timestamps.
SOLUSDT_1w: No missing timestamps

In [13]:
missing_df = find_missing_timestamps('BTCUSDT_1m_data')
if missing_df is not None and not missing_df.empty:
    missing_df.to_csv('BTCUSDT_1m_missing_timestamps.csv', index=False)

Found 3357 missing timestamps for BTCUSDT_1m_data.


In [14]:
df = pd.read_csv('data/exports/BTCUSDT_1m_data.csv', parse_dates=['timestamp'])

In [15]:
df.head()

,timestamp,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,...,4h_low,4h_close,1d_open,1d_high,1d_low,1d_close,1w_open,1w_high,1w_low,1w_close
0,2019-01-01 00:00:00,3701.23,3703.72,3701.09,3702.46,17.100110,1546300859999,63299.707312,180,5.746515,...,3675.04,3702.46,3701.23,3810.16,3642.0,3702.46,3803.12,4027.71,3630.33,3702.46
1,2019-01-01 00:01:00,3702.44,3702.63,3695.66,3697.04,23.700604,1546300919999,87681.084348,148,15.120491,...,3675.04,3697.04,3701.23,3810.16,3642.0,3697.04,3803.12,4027.71,3630.33,3697.04
2,2019-01-01 00:02:00,3699.42,3702.04,3696.08,3698.14,14.488615,1546300979999,53602.238977,80,12.700389,...,3675.04,3698.14,3701.23,3810.16,3642.0,3698.14,3803.12,4027.71,3630.33,3698.14
3,2019-01-01 00:03:00,3697.49,3698.19,3695.97,3696.51,8.499966,1546301039999,31423.283322,75,4.199726,...,3675.04,3696.51,3701.23,3810.16,3642.0,3696.51,3803.12,4027.71,3630.33,3696.51
4,2019-01-01 00:04:00,3697.20,3697.62,3695.00,3696.32,21.782886,1546301099999,80514.327585,93,15.080810,...,3675.04,3696.32,3701.23,3810.16,3642.0,3696.32,3803.12,4027.71,3630.33,3696.32


In [16]:
def insert_indicator_values(df):
    df['RSI'], df['RSI_MA'] = ti.calculate_rsi_with_ma(df['close'], rsi_period=14, ma_type="SMA", ma_length=14)
    df['MACD'], df['Signal'], df['MACD_Hist'] = ti.calculate_macd(df['close'], fast_period=12, slow_period=26, signal_period=9)
    df['BB_Mid'], df['BB_Upper'], df['BB_Lower'] = ti.calculate_bollinger_bands(df['close'], window=20, num_std=2)
    df['Stoch_K'], df['Stoch_D'] = ti.calculate_stochastic(df, k_period=14, d_period=3)

In [20]:
df_existing = pd.read_csv('data/exports/BTCUSDT_1d_data.csv')

In [21]:
df_existing.head()

,timestamp,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore,1w_open,1w_high,1w_low,1w_close
0,2017-08-17,4261.48,4485.39,4200.74,4285.08,795.150377,1503014399999,3.454770e+06,3427,616.248541,2.678216e+06,0,4261.48,4485.39,3850.0,4285.08
1,2017-08-18,4285.08,4371.52,3938.77,4108.37,1199.888264,1503100799999,5.086958e+06,5233,972.868710,4.129123e+06,0,4261.48,4485.39,3850.0,4108.37
2,2017-08-19,4108.37,4184.69,3850.00,4139.98,381.309763,1503187199999,1.549484e+06,2153,274.336042,1.118002e+06,0,4261.48,4485.39,3850.0,4139.98
3,2017-08-20,4120.98,4211.08,4032.62,4086.29,467.083022,1503273599999,1.930364e+06,2321,376.795947,1.557401e+06,0,4261.48,4485.39,3850.0,4086.29
4,2017-08-21,4069.13,4119.62,3911.79,4016.00,691.743060,1503359999999,2.797232e+06,3972,557.356107,2.255663e+06,0,4069.13,4453.91,3400.0,4016.00


In [24]:
def plot_fibonacci_chart(df, title="Fibonacci Retracement"):
    """Plot a candlestick chart with Fibonacci levels using Plotly."""
    fig = go.Figure()

    # Candlestick chart
    fig.add_trace(go.Candlestick(
        x=df['timestamp'],
        open=df['open'],
        high=df['high'],
        low=df['low'],
        close=df['close'],
        name="Candlesticks"
    ))

    # Add Swing Highs and Lows
    fig.add_trace(go.Scatter(
        x=df[df['swing_high']]['timestamp'],
        y=df[df['swing_high']]['high'],
        mode='markers',
        marker=dict(color='green', size=8, symbol='triangle-up'),
        name='Swing Highs'
    ))

    fig.add_trace(go.Scatter(
        x=df[df['swing_low']]['timestamp'],
        y=df[df['swing_low']]['low'],
        mode='markers',
        marker=dict(color='red', size=8, symbol='triangle-down'),
        name='Swing Lows'
    ))

    # Plot Fibonacci Levels if they exist
    fib_columns = ['fib_23_6', 'fib_38_2', 'fib_50', 'fib_61_8', 'fib_78_6', 'fib_100']
    colors = ['cyan', 'pink', 'orange', 'green', 'purple', 'blue']

    for level_name, color in zip(fib_columns, colors):
        if level_name in df.columns and df[level_name].notna().any():
            level_value = df[level_name].dropna().iloc[0]
            fig.add_trace(go.Scatter(
                x=[df['timestamp'].iloc[0], df['timestamp'].iloc[-1]],
                y=[level_value, level_value],
                mode='lines',
                line=dict(color=color, width=1, dash='dash'),
                name=f"{level_name}: {level_value:.2f}"
            ))

    # Chart layout
    fig.update_layout(
        title=title,
        xaxis_title="Time",
        yaxis_title="Price",
        legend_title="Legend",
        template="plotly_dark",
        height=600,
        width=1000
    )

    fig.update_xaxes(rangeslider_visible=False, type='date')
    fig.update_yaxes(tickprefix="$")

    fig.show()

In [25]:
df_with_fib = ti.calculate_fibonacci_from_swings(df_existing, window=5)

f:\Desktop\git\crypto_bi\business\utils\trading_indicators.py:290: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Downtrend' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[min_idx:max_idx, level] = value


In [26]:
df_with_fib.tail(40)

,timestamp,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,...,fib_100,fib_78_6,fib_61_8,fib_50,fib_38_2,fib_23_6,fib_0,swing_high_val,swing_low_val,trend
2700,2025-01-07,102235.60,102724.38,96181.81,96954.61,32059.875370,1736294399999,3.170422e+09,4480562,14815.114530,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2701,2025-01-08,96954.60,97268.65,92500.90,95060.61,33704.678940,1736380799999,3.205849e+09,5171208,15803.850910,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2702,2025-01-09,95060.61,95382.32,91203.67,92552.49,34544.836850,1736467199999,3.217670e+09,6027161,15323.626630,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2703,2025-01-10,92552.49,95836.00,92206.02,94726.11,31482.864240,1736553599999,2.962085e+09,5531444,15617.438230,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2704,2025-01-11,94726.10,95050.94,93831.73,94599.99,7047.904300,1736639999999,6.656860e+08,1496191,3412.376710,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2705,2025-01-12,94599.99,95450.10,93711.19,94545.06,8606.866220,1736726399999,8.132788e+08,1658863,4028.330310,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2706,2025-01-13,94545.07,95940.00,89256.69,94536.10,42619.564230,1736812799999,3.930666e+09,5740963,19473.880420,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2707,2025-01-14,94536.11,97371.00,94346.22,96560.86,27846.617530,1736899199999,2.676048e+09,4532923,14248.696160,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2708,2025-01-15,96560.85,100681.94,96500.00,100497.35,30509.991790,1736985599999,3.006242e+09,3487945,15494.178730,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2709,2025-01-16,100497.35,100866.66,97335.13,99987.30,27832.853170,1737071999999,2.765162e+09,3762755,13513.537070,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
plot_fibonacci_chart(df_with_fib, title="BTCUSDT - 1D Fibonacci Retracement")

# Archive

In [ ]:
"""
EXPORT HISTORY

symbol = "BTCUSDT"
interval = "1m"
start_time = int(pd.Timestamp("2019-01-01").timestamp() * 1000)
export_historical_data(symbol, interval, start_time)

symbol = "ETHUSDT"
interval = "1m"
start_time = int(pd.Timestamp("2019-01-01").timestamp() * 1000)
export_historical_data(symbol, interval, start_time)

symbol = "SOLUSDT"
interval = "1m"
start_time = int(pd.Timestamp("2019-01-01").timestamp() * 1000)
export_historical_data(symbol, interval, start_time)

symbol = "ICPUSDT"
interval = "1m"
start_time = int(pd.Timestamp("2019-01-01").timestamp() * 1000)
export_historical_data(symbol, interval, start_time)

symbol = "AVAXUSDT"
interval = "1m"
start_time = int(pd.Timestamp("2019-01-01").timestamp() * 1000)
export_historical_data(symbol, interval, start_time)
"""